In [4]:
%pip install google-generativeai
%pip install openai
import google.generativeai as genai
import openai
import getpass


In [ ]:
print("Enter your Gemini API key: Please Note this is a demo version, only Gemini is currently supported")
API_KEY = getpass.getpass()
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
#response = model.generate_content("Write a story about a magic backpack")
#print(response.text)

In [70]:
def chatbot_comm(user_input, history=[]):
    chat = model.start_chat(
    history = history
    )
    generation_config = {
        "temperature": 0.1  # Adjust temperature for more deterministic (lower) or more creative (higher) responses
    }
    response = chat.send_message(user_input, generation_config=generation_config)
    history.append({
        "role": "user",
        "parts": [{"text": user_input}]
    })
    history.append({
        "role": "model",
        "parts": [{"text": response.text}]
    })
    return response.text, history

# Moderation is not working due to exceeded limit. Will implement later once billing is sorted.
# def moderate_response(response):
#     url = "https://api.openai.com/v1/moderations"
#     from openai import OpenAI
#     client=OpenAI(api_key=api_key)
#     response = client.moderations.create(input=response)
#     return response.results[0].flagged

def collect_info(user_input, history=[]):
    sys_message2 = """
    Remember to ask the questions one by one and wait for the user's response before asking the next question.
    Keep asking them questions to understand their needs better. The Following data needs to be collected: 
    1. User's budget 
    2. GPU Intensity 
    3. Multitasking 
    4. Portability
    5. Processing Speed
    6. Display Quality
    The Values for the above parameters should only be Low, Medium, High. Only Budget can be a number in INR. 
    You need to infer the values from the user's input as shown in the example below:
    -- GPU Intensity : High
    -- Multitasking : Medium
    -- Portability : Low
    -- Processing Speed : High
    -- Display Quality : High
    -- Budget : 50000
    Keep asking questions until you have all the information you need. 
    in the response, do not include the above instructions or response.
    If 'Not Done', continue asking questions.
    if 'Done', display the collected information in the above format
    Mandatorily respond with 'Done' or 'Not Done' after the response.
    """
    combined_input = f"{sys_message2}\n\nUser: {user_input}"
    response, history = chatbot_comm(combined_input, history)
    print("bot:", response.split('\n')[0])
    print("-"*20)
    # user_input = input()
    # print("user:", user_input)
    while 'Not Done' in response:
        user_input = input()
        print("user:", user_input)
        response, history = chatbot_comm(user_input, history)
        print("bot:", response.split('\n')[0])
        print("-"*20)
        #print('test 1 ------------------------------')
    print(response.split('\n')[0])
    return response

def display_info(data):
    sys_message3 = """
    You are the most intelligent laptop recommendation assistant.
    The information collected from the user is as follows:
    {data}
    Use this information to recommend the best laptop for the user.
    respond with the name of the laptop, its price, and and a small description of the laptop.
    Respond with three best recommendations.
    """
    response, history = chatbot_comm(sys_message3)
    print("bot:", response)
    print("-"*20)
    


def initialize_conversation():
    print("Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. ")
    sys_message = """You are a helpful assistant that can help user choose the best laptop for their needs. 
    Greet the user and ask how you can help them.
    """
    #combined_input = f"{sys_message}\n\nUser: {user_input}"
    response, history = chatbot_comm(sys_message)
    print("Bot: ", response)
    user_input = input()
    print("You: ", user_input)
    #mod_resp = moderate_response(response)
    #print(mod_resp)
    # print("Bot: ", response)
    # user_input = input()
    # print("You: ", user_input)
    # response, history = chatbot_comm(user_input, history)
    # print("Bot: ", response)
    data = collect_info(user_input, history) 
    display_info(data)
    #print(history)


initialize_conversation()



Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. 
Bot:  Hi there! 👋  I'm here to help you find the perfect laptop.  What are you looking for in a laptop? Tell me about your needs and I'll do my best to guide you to the right choice. 😊 

You:  I am looking for gaming laptop
Okay, a gaming laptop! That's great.  To help me find the perfect one for you, can you tell me your budget in INR? 
Not Done 

bot: Okay, a gaming laptop! That's great.  To help me find the perfect one for you, can you tell me your budget in INR? 
Okay, a gaming laptop! That's great.  To help me find the perfect one for you, can you tell me your budget in INR? 
Not Done 

user: 


ValueError: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.